In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.filterwarnings('ignore')


import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split


from sklearn.metrics import log_loss,roc_auc_score,precision_score,f1_score,recall_score,roc_curve,auc
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score,fbeta_score,matthews_corrcoef
from sklearn import metrics


from sklearn.model_selection import StratifiedKFold


from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.ensemble import RandomForestClassifier,VotingClassifier,AdaBoostClassifier,GradientBoostingClassifier,RandomForestClassifier,ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC 


from scipy import stats

In [ ]:
# create a base path
BASE_PATH = "../input/treatement-prediction-of-employees/"        

# read the Training.csv and Test.csv files into dataframes
dt = pd.read_csv(BASE_PATH + 'training_.csv')
test_dataframe = pd.read_csv(BASE_PATH + 'test.csv')

In [ ]:
dt.head()

In [ ]:
print(dt.shape)

### 3.Data Analysing & Preprocessing:

In [ ]:
 #Checking Imbalance data:
dt['treatment'].value_counts()

In [ ]:
dt.columns

In [ ]:
dt['Country'].value_counts()

In [ ]:
dt['leave'].value_counts()

In [ ]:
dt.isna().sum()

In [ ]:
dt.info()

### 4. Exploratory Data Analysis:

->Checking distribution of 'Treatment'(Target/dependent variable)

In [ ]:
# Plotting attrition of employees:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharey=False, figsize=(14,6))

ax1 = dt['treatment'].value_counts().plot.pie( x="health reatment" ,y ='no.of employees', 
                   autopct = "%1.0f%%",labels=["Health Condition","Normal"], startangle = 60,ax=ax1);
ax1.set(title = 'Percentage of employees in Dataset')

ax2 = dt["treatment"].value_counts().plot(kind="barh" ,ax =ax2)
for i,j in enumerate(dt["treatment"].value_counts().values):
    ax2.text(.5,i,j,fontsize=12)
ax2.set(title = 'No. of employess require treatment as per the Data')

->Age Distribution

In [ ]:

sns.distplot(dt['Age'].dropna(),kde=False,color='darkred',bins=40)

In [ ]:

sns.set_style('darkgrid')
sns.countplot(x='work_interfere',hue='mental_health_consequence',data=dt,palette='viridis')

In [ ]:
sns.set_style('darkgrid')
sns.countplot(x='work_interfere',hue='phys_health_consequence',data=dt,palette='viridis')

In [ ]:
dt.Gender.unique()

In [ ]:
dt.work_interfere.unique()

In [ ]:
# Summary statistics of categorical columns:
dt.describe(include =[np.object])

In [ ]:
# Summary statistics of numerical columns:
dt.describe(include =[np.number])

In [ ]:
#Replacing gender names with three commonly used labels,this will reduce the dimensionality while encoding procedure.

dt['Gender']=dt['Gender'].replace(['Female','M','Male','male','female','m','Male-ish','maile',
                                   'Trans-female','Cis Female','F','something kinda male?',
                                   'Cis Male','Woman','f','Mal','Male (CIS)','queer/she/they',
                                   'non-binary','Femake','woman','Make','Nah','All','Enby',
                                   'fluid','Genderqueer','Female','Androgyne','Agender',
                                   'cis-female/femme','Guy (-ish) ^_^','male leaning androgynous',
                                   'Male','Man','Trans woman','msle','Neuter','Female (trans)',
                                   'queer','Female (cis)','Mail','cis male','A little about you',
                                   'Malr'],
                                  ['female','male','male','male','female','male','transgender','transgender',
  'transgender','transgender','female','transgender',
  'transgender','female','female','male','transgender','transgender',
  'transgender','transgender','female','transgender','transgender','transgender','transgender',
  'transgender','transgender','female','transgender','transgender',
  'transgender','transgender','transgender',
  'male','male','transgender','transgender','transgender','transgender',
  'transgender','transgender','transgender','transgender','transgender',
 'male'],inplace=False)
                        

In [ ]:
dt.Gender.unique() 

In [ ]:
dt['Gender']=dt['Gender'].replace(['Female ', 'Male '],['female','male'],inplace=False)

In [ ]:
dt.Gender.unique()

In [ ]:
sns.set_style('dark')
sns.countplot(x='mental_health_interview',hue='Gender',data=dt,palette='RdBu_r')

In [ ]:
dt.shape

In [ ]:

dt.drop(['S.No','self_employed','Country','Timestamp','no_employees','state','comments'],axis='columns',inplace=True)

In [ ]:
dt.head()

In [ ]:
dt.shape

In [ ]:
age_mode=dt['Age'].mode()
age_mode

In [ ]:
dt['Age']=dt['Age'].replace([99999999999,-1726,329,-29],[29,29,29,29],inplace=False)
 

In [ ]:
dt['Age'].value_counts()

In [ ]:

sns.distplot(dt['Age'].dropna(),kde=False,color='darkred',bins=40)

From the above plot we can say that Age is now distributed normally ,before it was totatlly left skewed. As the outliers are removed the distribution looks perfect for further processing.

In [ ]:
dt['treatment']=dt['treatment'].replace(['Yes','No'],[1,0],inplace=False)

In [ ]:
dt['work_interfere']=dt['work_interfere'].fillna(value='Nan',inplace=False)

In [ ]:
dt['work_interfere'].unique()

In [ ]:
dt.head()


In [ ]:
dt.shape

In [ ]:
dt.isna().sum()

In [ ]:
## Encoding categorical variables:
dt= pd.get_dummies(dt, drop_first=True)

dt.head()

In [ ]:
dt.shape

In [ ]:
# segregating dataset into features i.e., X and target variables i.e., y
X = dt.drop(['treatment'],axis=1)
y = dt['treatment']

In [ ]:
X.shape

In [ ]:
X.head()

### 5.Train & Test split of given data:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2,shuffle=True, random_state=5)

In [ ]:
print('Training Set')
print(X_train.shape)
print(y_train.shape)

print('Test Set')
print(X_test.shape)
print(y_test.shape)

### 6. Checking Feature correlation & Importance

In [ ]:
### constant features check and removal if any using Filter Method: 
from sklearn.feature_selection import VarianceThreshold
constant_filter = VarianceThreshold(threshold=0)
constant_filter.fit(X_train)

In [ ]:
constant_filter.get_support().sum()

In [ ]:
constant_list=[not temp for temp in constant_filter.get_support()]
constant_list

In [ ]:
corrmat= X_train.corr()

In [ ]:
plt.figure(figsize=(20,5))

dataplot=sns.heatmap((corrmat),annot= True ) 
plt.show()

In [ ]:
def get_correlation(da ,threshold):
    corr_col= set()
    corrmat= dt.corr()
    for i in range(len(corrmat.columns)):
        for j in range(i):
            if abs(corrmat.iloc[i,j])>threshold:
                colname=corrmat.columns[i]
                corr_col.add(colname)
                
   
    return corr_col 

In [ ]:
corr_features=get_correlation(X_train,0.85)
corr_features

In [ ]:
len(corr_features)

In [ ]:
X_train_uncorr = X_train.drop(labels=corr_features,axis=1)
X_test_uncorr= X_test.drop(labels=corr_features,axis=1)

In [ ]:
X_train_uncorr.shape,X_test_uncorr.shape

In [ ]:
#Feature grouping and feature importance:
#vertical stacking
corrmat

In [ ]:
corrdata=corrmat.abs().stack()
corrdata

In [ ]:
corrdata = corrdata.sort_values(ascending=False)
corrdata

In [ ]:
corrdata=corrdata[corrdata>0.85]
corrdata=corrdata[corrdata<1]
corrdata

In [ ]:
corrdata=pd.DataFrame(corrdata).reset_index()
corrdata.columns=['features1','features2','corr_values']
corrdata

In [ ]:
#Grouping of these corelated features and finding  feature imp for each group,will select only one feature from each group and discard other feature.
#creating a list for group fetaure list
grouped_feature_list=[]
correlated_groups_list=[]
for feature in corrdata.features1.unique():
    if feature not in grouped_feature_list:
        correlated_block=corrdata[corrdata.features1==feature]
        grouped_feature_list=grouped_feature_list+list(correlated_block.features2.unique())+[feature]
        correlated_groups_list.append(correlated_block)

In [ ]:
len(correlated_groups_list)

In [ ]:
#feature importance based on tree based classifiers:
important_features=[]
for group in correlated_groups_list:
    features=list(group.features1.unique())+list(group.features2.unique())
    rf= RandomForestClassifier(n_estimators=100,random_state=0)
    rf.fit(X_train[features],y_train)
    
    importance=pd.concat([pd.Series(features),pd.Series(rf.feature_importances_)],axis=1)
    importance.columns=['features','importance']
    importance.sort_values(by='importance',ascending=False,inplace=True)
    feat1=importance.iloc[0]
    important_features.append(feat1)

In [ ]:
important_features


In [ ]:
important_features_et=[]
for group in correlated_groups_list:
    features=list(group.features1.unique())+list(group.features2.unique())
    et= ExtraTreesClassifier(n_estimators=100,random_state=0)
    et.fit(X_train[features],y_train)
    
    importance=pd.concat([pd.Series(features),pd.Series(et.feature_importances_)],axis=1)
    importance.columns=['features','importance']
    importance.sort_values(by='importance',ascending=False,inplace=True)
    feat2=importance.iloc[0]
    important_features_et.append(feat2)

In [ ]:
## Checking distribution of target variable in train test split:
print('Distribution of target variable in training set:')
print(y_train.value_counts())

print('Distribution of target variable in test set:')
print(y_test.value_counts())

In [ ]:
print(X_test_uncorr.shape)

## a.Feature Normalization:

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train[['Age']] = scaler.fit_transform(X_train[['Age']])
X_train.head()

In [ ]:
X_test[['Age']] = scaler.fit_transform(X_test[['Age']])
X_test.head()

 ## 7.Cross-Validation of Models:

In [ ]:
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
import xgboost as xgb

# function initializing baseline machine learning models
#Return statement is used to end the execution of the function call and returns the result to the caller i.e., 
#value of the expression following the return keyword.
def GetBasedModel():
    basedModels = []
    basedModels.append(('LR_L2'   , LogisticRegression(penalty='l2')))
    basedModels.append(('KNN7'  , KNeighborsClassifier(7)))
    basedModels.append(('KNN5'  , KNeighborsClassifier(5)))
    basedModels.append(('KNN9'  , KNeighborsClassifier(9)))
    basedModels.append(('KNN11'  , KNeighborsClassifier(11)))
    basedModels.append(('CART' , DecisionTreeClassifier()))
    basedModels.append(('NB'   , GaussianNB()))
    basedModels.append(('SVM RBF'  , SVC(kernel='rbf',gamma='auto',probability=True)))
    basedModels.append(('AB'   , AdaBoostClassifier()))
    basedModels.append(('GBM'  , GradientBoostingClassifier(n_estimators=100,max_features='sqrt')))
    basedModels.append(('RF_Ent100'   , RandomForestClassifier(criterion='entropy',n_estimators=100)))
    basedModels.append(('RF_Ent500'   , RandomForestClassifier(criterion='entropy',n_estimators=500)))
    basedModels.append(('RF_Gini100'   , RandomForestClassifier(criterion='gini',n_estimators=100)))
    basedModels.append(('ET100'   , ExtraTreesClassifier(n_estimators= 100)))
    basedModels.append(('ET500'   , ExtraTreesClassifier(n_estimators= 500)))
    basedModels.append(('MLP', MLPClassifier()))
    basedModels.append(('XGB_2000', xgb.XGBClassifier(n_estimators= 2000)))
    basedModels.append(('XGB_500', xgb.XGBClassifier(n_estimators= 500)))
    basedModels.append(('XGB_100', xgb.XGBClassifier(n_estimators= 100)))
    basedModels.append(('XGB_1000', xgb.XGBClassifier(n_estimators= 1000)))
    basedModels.append(('ET1000'   , ExtraTreesClassifier(n_estimators= 1000)))
    
    return basedModels

# function for performing 10-fold cross validation of all the baseline models
def BasedLine2(X_train, y_train,models):
    # Test options and evaluation metric
    num_folds = 10
    scoring = 'accuracy'
    seed = 7
    results = []
    names = []
    for name, model in models:
        kfold = model_selection.KFold(n_splits=10 ,random_state=seed, shuffle=True)
        cv_results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
        results.append(cv_results)
        names.append(name)
        msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
        print(msg)
         
        
    return results,msg

In [ ]:
models = GetBasedModel()
names,results = BasedLine2(X_train, y_train,models)

**Choosing "SVM Linear" as it gave 0.827008 as result.**

## Model Building:

In [ ]:
svc = SVC(kernel='linear',gamma='auto',probability=True)


## Chained it at runtime:

In [ ]:
from sklearn.pipeline import make_pipeline

In [ ]:
pipe=make_pipeline(scaler,svc)

In [ ]:
pipe.fit(X_train,y_train)

In [ ]:
X_test.head()

In [ ]:
y_pred=pipe.predict(X_test)
y_pred

# a.Checking Accuracies of the selected Support Vector Machine:

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

In [ ]:
from sklearn.metrics import recall_score
recall_score(y_test,y_pred)

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test,y_pred)

## 10. Model Evaluation   

 In this step we will first define which evaluation metrics we will use to evaluate our model. The most important evaluation metric for this problem domain is **sensitivity, specificity, Precision, F1-measure, Geometric mean and mathew correlation coefficient and finally ROC AUC curve**

In [ ]:
CM=confusion_matrix(y_test,y_pred)
sns.heatmap(CM, annot=True)

TN = CM[0][0]
FN = CM[1][0]
TP = CM[1][1]
FP = CM[0][1]
specificity = TN/(TN+FP)
loss_log = log_loss(y_test, y_pred)
acc= accuracy_score(y_test, y_pred)
roc=roc_auc_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

mathew = matthews_corrcoef(y_test, y_pred)
model_results =pd.DataFrame([['SVM',acc, prec,rec,specificity, f1,roc, loss_log,mathew]],
               columns = ['Model', 'Accuracy','Precision', 'Sensitivity','Specificity', 'F1 Score','ROC','Log_Loss','mathew_corrcoef'])

model_results
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

## a.ROC AUC Curve

In [ ]:
def roc_auc_plot(y_true, y_proba, label=' ', l='-', lw=1.0):
    from sklearn.metrics import roc_curve, roc_auc_score
    fpr, tpr, _ = roc_curve(y_true, y_proba[:,1])
    ax.plot(fpr, tpr, linestyle=l, linewidth=lw,
            label="%s (area=%.3f)"%(label,roc_auc_score(y_true, y_proba[:,1])))

f, ax = plt.subplots(figsize=(12,8))


roc_auc_plot(y_test,svc.predict_proba(X_test),label='Support Vector Machine ',l='-')


ax.plot([0,1], [0,1], color='k', linewidth=0.5, linestyle='--', 
        )    
ax.legend(loc="lower right")    
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_xlim([0, 1])
ax.set_ylim([0, 1])
ax.set_title('Receiver Operator Characteristic curves')
sns.despine()

## b.Precision-Recall Curve

In [ ]:
def precision_recall_plot(y_true, y_proba, label=' ', l='-', lw=1.0):
    from sklearn.metrics import precision_recall_curve, average_precision_score
    precision, recall, _ = precision_recall_curve(y_test,
                                                  y_proba[:,1])
    average_precision = average_precision_score(y_test, y_proba[:,1],
                                                     average="micro")
    ax.plot(recall, precision, label='%s (average=%.3f)'%(label,average_precision),
            linestyle=l, linewidth=lw)

f, ax = plt.subplots(figsize=(12,8))

precision_recall_plot(y_test,svc.predict_proba(X_test),label='Support Vector Machine ',l='-')


ax.plot([0,1], [0,1], color='k', linewidth=0.5, linestyle='--',)
        
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.legend(loc="lower left")
ax.grid(True)
ax.set_xlim([0, 1])
ax.set_ylim([0, 1])
ax.set_title('Precision-recall curves')
sns.despine()

## 11. Save Model using 'Pickle'

In [ ]:
import pickle
output=open("TreatmentPred.pickle","wb")

In [ ]:
pickle.dump(pipe,output)

## Testing the given "test" data by importing in a new notebook with a loaded pipeline in Pickle serialization dump .

In [ ]:
import numpy as np
import pandas as pd

### Importing test.csv


In [ ]:
dt_new = pd.read_csv(BASE_PATH + 'test.csv')

In [ ]:
dt_new.head()

In [ ]:
dt_new.columns

In [ ]:
 dt_new['Age'].value_counts()

In [ ]:
dt_new.isna().sum()

In [ ]:
dt_new.Gender.unique()

In [ ]:
dt_new['Gender']=dt['Gender'].replace(['Male', 'female', 'Female', 'M', 'male', 'Male ', 'm', 'p', 'F',
       'Woman', 'femail', 'Cis Man',
       'ostensibly male, unsure what that really means', 'f'],
    ['male','female','female','male','male','male','male','transgender','female','female','female','transgender','transgender',
    'female'],inplace=False)

In [ ]:
dt_new.Gender.unique()

In [ ]:
dt_new.head()

## -> In the above Test table 'Treatment' column is not provided, which will be predicted from the below steps.

In [ ]:
age_mode=dt_new['Age'].mode()
age_mode

In [ ]:
dt_new['Age']=dt_new['Age'].replace([-1],[32],inplace=False)

In [ ]:
 dt_new['Age'].value_counts()

Outlier '-1' has been removed from 'Age' column of Test.

-> Below ,making the same changes(Preprocessing) to 'Test data' that have been made for the 'Training_' data.

In [ ]:
dt_new['work_interfere'].value_counts()

In [ ]:
dt_new['work_interfere']=dt_new['work_interfere'].fillna(value='Nan',inplace=False)

In [ ]:
dt_new['work_interfere'].unique()

In [ ]:
dt_new['work_interfere'].value_counts()

In [ ]:
dt_new.head()

In [ ]:
dt_new.drop(['S.No','self_employed','Country','Timestamp','no_employees','state','comments'],inplace=True,axis=1)

In [ ]:
dt_new.head()

In [ ]:
dt_new= pd.get_dummies(dt_new, drop_first=True)

dt_new.head()
dt_new.shape

In [ ]:
import pickle

In [ ]:
model=open("TreatmentPred.pickle","rb")

In [ ]:
employee_model=pickle.load(model)

In [ ]:
employee_model.predict(dt_new)

In [ ]:
pred=employee_model.predict(dt_new)

In [ ]:
dt_new['treatment']=pred

In [ ]:
dt_new.head()

## ->Above 'Test' table has been inserted with the predicted 'Treatment' columns.